In [ ]:
import numpy as np
import scipy.sparse as sparse
import pandas as pd
import matplotlib.pyplot as plt
import demes, msprime, demesdraw
import tspop, tskit, tstrait
import cython
import statsmodels.api as sm
%load_ext cython

plt.rcParams["font.family"] = "Arial"

In [ ]:
%%cython --cplus

import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
def get_lanc_dense(
    const long[:] seg_left, # left of segment
    const long[:] seg_right, # right of segment
    const long[:] seg_lanc, # lanc of segment
    const long[:] node_ptr, # similar to sparse mat indptr
    const long[:] site_pos, # position of sites
    int n_site, # number of sample nodes
    int n_node # number of sites
):
    
    cdef long[::1,:] lanc_dense = np.empty((n_site, n_node), dtype=long, order='F')

    cdef Py_ssize_t i_node, i_site, 
    cdef Py_ssize_t node_begin, node_end, pos_site
    cdef Py_ssize_t seg_begin, seg_end
    with nogil:
        for i_node in range(n_node):
            node_begin, node_end = node_ptr[i_node], node_ptr[i_node+1]
            for i_site in range(n_site):
                pos_site = site_pos[i_site]
                for i_seg in range(node_begin, node_end):
                    seg_begin, seg_end = seg_left[i_seg], seg_right[i_seg]
                    if (pos_site >= seg_begin) and (pos_site < seg_end):
                        lanc_dense[i_site, i_node] = seg_lanc[i_seg]

    return np.asarray(lanc_dense)

In [ ]:
# from https://github.com/tskit-dev/tskit/issues/504
def allele_frequencies(ts, sample_sets=None):
    if sample_sets is None:
       sample_sets = [ts.samples()] 
    n = np.array([len(x) for x in sample_sets])
    def f(x):
       return x / n
    return ts.sample_count_stat(sample_sets, f, len(sample_sets), windows='sites', polarised=True, mode='site', strict=False, span_normalise=False)

# convert site x node to site x individual matrix
def hap_to_dip(mat): # assumes diploid
    n_site, n_node = mat.shape
    n_individual = int(n_node/2)

    data = np.ones(n_node)
    indices = np.repeat(np.arange(n_individual),2)
    indptr = np.arange(n_node+1)
    
    sum_mat = sparse.csr_matrix((data,indices,indptr), shape=(n_node, n_individual))
    return(mat @ sum_mat)

In [ ]:
model = demes.load('amr_demes.yml')
ax = demesdraw.tubes(model, log_time=True)
plt.savefig('history.svg')
plt.show()

In [ ]:
demography = msprime.Demography.from_demes(model)
# 12 is start_time=300 divided by generation time=25
demography.add_census(time=12.01) # census required to track ancestors
demography.sort_events() # event should be sorted after adding census
samples = {'AFR':5000, 'EUR':5000, 'AMR': 10000}
# rates from https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_HomSap
# chromosome 22
seq_length = 50818468
ts = msprime.sim_ancestry(
    [msprime.SampleSet(n, ploidy=2, population=p) for p, n in samples.items()],
    demography=demography,
    model=[
        msprime.DiscreteTimeWrightFisher(duration=5),
        msprime.StandardCoalescent(),
    ],
    recombination_rate=2.10572e-08,
    sequence_length=seq_length,
    record_migrations=True,
    random_seed=1
)
mts = msprime.sim_mutations(ts, rate=1.29e-08, random_seed=1)

In [ ]:
af = allele_frequencies(mts, [mts.samples(0), mts.samples(1)])
af_afr, af_eur = af[:,0], af[:,1]
af_threshold = 0.01
rare_variants = (af_afr < af_threshold) | (af_eur < af_threshold)

In [ ]:
mts_common = mts.delete_sites(np.arange(mts.num_sites)[rare_variants])

In [ ]:
gt_afr = mts_common.genotype_matrix(samples=mts_common.samples(0))
gt_eur = mts_common.genotype_matrix(samples=mts_common.samples(1))
gt_amr = mts_common.genotype_matrix(samples=mts_common.samples(2))

In [ ]:
pa = tspop.get_pop_ancestry(mts_common, census_time=12.01) # admixture happend in t=12, census time t=10.01
pa_amr = pa.subset_tables(subset_samples=mts_common.samples(2))[1] # subset AMR

In [ ]:
%%time
seg_left = pa_amr.left.values.astype(int)
seg_right = pa_amr.right.values.astype(int)
seg_lanc = pa_amr.population.values.astype(int)
node_ptr = np.zeros(gt_amr.shape[1]+1, dtype=int) # similar to sparse matrix pointer
node_ptr[1:] = pa_amr['sample'].value_counts(sort=False).cumsum().values
site_pos = mts_common.tables.sites.position.astype(int)
n_site, n_node = gt_amr.shape

lanc_dense = get_lanc_dense(seg_left, seg_right, seg_lanc, node_ptr, site_pos, n_site, n_node)

In [ ]:
gt_amr_afr = gt_amr * (1-lanc_dense) 
gt_amr_eur = gt_amr * lanc_dense

In [ ]:
gt_afr_dip = hap_to_dip(gt_afr)
gt_eur_dip = hap_to_dip(gt_eur)
gt_amr_dip = hap_to_dip(gt_amr)

gt_amr_afr_dip = hap_to_dip(gt_amr_afr) 
gt_amr_eur_dip = hap_to_dip(gt_amr_eur)

lanc_dip = hap_to_dip(lanc_dense)

In [ ]:
num_sites = mts_common.num_sites

In [ ]:
polygenicity_props = [1e-3, 5e-3, 1e-2, 5e-2]

afr_list = []
eur_list = []
amr_afr_list = [] 
amr_eur_list = []
sim_result_list = []
for polygenicity_prop in polygenicity_props:
    
    # simulate trait
    trait_model = tstrait.trait_model(distribution="normal", mean=0, var=1)
    sim_result = tstrait.sim_phenotype(
      ts=mts_common, num_causal=int(num_sites * polygenicity_prop), model=trait_model, h2=0.5, random_seed=1
    )
    sim_result_list.append(sim_result)
    
    pheno_afr = sim_result.phenotype.iloc[:5000,:]
    pheno_eur = sim_result.phenotype.iloc[5000:10000,:]
    pheno_amr = sim_result.phenotype.iloc[10000:,:]

    # get true estimate 
    beta_afr = np.zeros(num_sites)
    beta_eur = np.zeros(num_sites)
    beta_amr_afr = np.zeros(num_sites)
    beta_amr_eur = np.zeros(num_sites)
    
    for i in range(num_sites):
        # afr
        exog_afr = gt_afr_dip[i,:]
        endog_afr = pheno_afr.genetic_value.values
        fit_afr = sm.OLS(endog_afr, sm.add_constant(exog_afr)).fit().params
        if len(fit_afr) > 1:
            beta_afr[i] = fit_afr[1]
        
        # eur
        exog_eur = gt_eur_dip[i,:]
        endog_eur = pheno_eur.genetic_value.values
        fit_eur = sm.OLS(endog_eur, sm.add_constant(exog_eur)).fit().params
        if len(fit_eur) > 1:
            beta_eur[i] = fit_eur[1]
            
        # amr
        exog_afr = gt_amr_afr_dip[i,:]
        exog_eur = gt_amr_eur_dip[i,:]
        lanc = lanc_dip[i,:]
        exog = np.vstack((exog_afr, exog_eur, lanc)).T
        endog = pheno_amr.genetic_value.values
        fit = sm.OLS(endog, sm.add_constant(exog)).fit().params
        beta_amr_afr[i] = fit[1]
        beta_amr_eur[i] = fit[2]

    afr_list.append(beta_afr)
    eur_list.append(beta_eur)
    amr_afr_list.append(beta_amr_afr)
    amr_eur_list.append(beta_amr_eur)

In [ ]:
from scipy.stats import gaussian_kde

fig, ax = plt.subplots(2, 4, figsize=(17,8))

for col in range(4):

    site_causal = np.random.binomial(n=1, p=0.2, size=num_sites).astype(bool)
    
    x, y = afr_list[col][site_causal], amr_afr_list[col][site_causal]
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    ax[0, col].scatter(x, y, c=z, s=5, rasterized=True)

    x, y = eur_list[col][site_causal], amr_eur_list[col][site_causal]
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    ax[1, col].scatter(x, y, c=z, s=5, rasterized=True)

    lim= 2e-3 / np.sqrt(polygenicity_props[col]) * 1.1
    ax[0,col].set_xlim([-lim,lim])
    ax[0,col].set_ylim([-lim,lim])
    ax[1,col].set_xlim([-lim,lim])
    ax[1,col].set_ylim([-lim,lim])
    ax[0,col].plot([-1,1],[-1,1],ls='--', color='black')
    ax[1,col].plot([-1,1],[-1,1],ls='--', color='black')


    afr_cor = np.corrcoef(afr_list[col], amr_afr_list[col])[0,1]
    eur_cor = np.corrcoef(eur_list[col], amr_eur_list[col])[0,1]
    ax[0,col].text(0.1, 0.9, r'$r_{g,all} = %.2f$' % afr_cor, fontsize=15, transform=ax[0,col].transAxes)
    ax[1,col].text(0.1, 0.9, r'$r_{g,all} = %.2f$' % eur_cor, fontsize=15, transform=ax[1,col].transAxes)

    z = np.log10(polygenicity_props[col])
    pow, val = math.floor(z), int(10**(z-math.floor(z)) + 0.5)
    ax[0,col].set_title(r'Causal proportion: $%d \times 10^{%d}$' % (val, pow), fontsize=15)
    
    ax[0,col].locator_params(axis='both', nbins=5)
    ax[1,col].locator_params(axis='both', nbins=5)

ax[0,0].set_ylabel('African', fontsize=15)
ax[1,0].set_ylabel('European', fontsize=15)

fig.text(0.5, -0.02, 'Single-continental marginal effect size', fontsize=15, ha='center', va='center')
fig.text(-0.02, 0.5, 'Admixed TRACTOR effect size', fontsize=15, ha='center', va='center', rotation=90)


plt.tight_layout()

plt.savefig('full.svg', bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams["mathtext.fontset"] = 'cm'

from scipy.stats import gaussian_kde

fig, ax = plt.subplots(2, 4, figsize=(17,8))

for col in range(4):

    site_causal = sim_result_list[col].effect_size.site_id
    
    x, y = afr_list[col][site_causal], amr_afr_list[col][site_causal]
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    ax[0, col].scatter(x, y, c=z, s=10, rasterized=True)

    x, y = eur_list[col][site_causal], amr_eur_list[col][site_causal]
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    ax[1, col].scatter(x, y, c=z, s=10, rasterized=True)

    lim= 2e-3 / np.sqrt(polygenicity_props[col]) * 1.1
    ax[0,col].set_xlim([-lim,lim])
    ax[0,col].set_ylim([-lim,lim])
    ax[1,col].set_xlim([-lim,lim])
    ax[1,col].set_ylim([-lim,lim])
    ax[0,col].plot([-1,1],[-1,1],ls='--', color='black')
    ax[1,col].plot([-1,1],[-1,1],ls='--', color='black')


    afr_cor = np.corrcoef(afr_list[col][site_causal], amr_afr_list[col][site_causal])[0,1]
    eur_cor = np.corrcoef(eur_list[col][site_causal], amr_eur_list[col][site_causal])[0,1]
    ax[0,col].text(0.1, 0.9, r'$r_{g,causal} = %.2f$' % afr_cor, fontsize=15, transform=ax[0,col].transAxes)
    ax[1,col].text(0.1, 0.9, r'$r_{g,causal} = %.2f$' % eur_cor, fontsize=15, transform=ax[1,col].transAxes)

    z = np.log10(polygenicity_props[col])
    pow, val = math.floor(z), int(10**(z-math.floor(z)) + 0.5)
    ax[0,col].set_title(r'Causal proportion: $%d \times 10^{%d}$' % (val, pow), fontsize=15)

    ax[0,col].locator_params(axis='both', nbins=5)
    ax[1,col].locator_params(axis='both', nbins=5)

ax[0,0].set_ylabel('African', fontsize=15)
ax[1,0].set_ylabel('European', fontsize=15)

fig.text(0.5, -0.02, 'Single-continental marginal effect size', fontsize=15, ha='center', va='center')
fig.text(-0.02, 0.5, 'Admixed TRACTOR effect size', fontsize=15, ha='center', va='center', rotation=90)

plt.tight_layout()
plt.savefig('causal.svg', bbox_inches='tight')
plt.show()

In [ ]:
coef_afr = np.zeros(num_sites)
coef_eur = np.zeros(num_sites)
for i in range(num_sites):
    lanc = lanc_dip[i,:]
    endog_afr = gt_amr_afr_dip[i,:]
    endog_eur = gt_amr_eur_dip[i,:]

    coef_afr[i] = np.dot(2-lanc, endog_afr) / np.dot(2-lanc,2-lanc)
    coef_eur[i] = np.dot(lanc, endog_eur) / np.dot(lanc,lanc)

In [ ]:
af_common = allele_frequencies(mts_common, [mts_common.samples(0), mts_common.samples(1)])

In [ ]:
afr_resid = gt_amr_afr_dip - (2-lanc_dip) * coef_afr[:,None]
eur_resid = gt_amr_eur_dip - lanc_dip * coef_eur[:,None]

In [ ]:
from scipy.spatial.distance import pdist, squareform

n_loci = 5000
site_sub = np.linspace(0, 69700, n_loci).astype(int)
afr_resid_sub = afr_resid[site_sub,:]
eur_resid_sub = eur_resid[site_sub,:]
afr_sub = gt_afr_dip[site_sub,:]
eur_sub = gt_eur_dip[site_sub,:]
gt_amr_afr_sub = gt_amr_afr_dip[site_sub,:]
gt_amr_eur_sub = gt_amr_eur_dip[site_sub,:]
site_distance = squareform(pdist(mts_common.tables.sites[site_sub].position[:,None], 'cityblock'))

In [ ]:
afr_ld = np.nan_to_num(1/10000 * (afr_resid_sub @ gt_amr_afr_sub.T) / np.outer(afr_resid_sub.std(axis=1), gt_amr_afr_sub.std(axis=1)))
eur_ld = np.nan_to_num(1/10000 * (eur_resid_sub @ gt_amr_eur_sub.T) / np.outer(eur_resid_sub.std(axis=1), gt_amr_eur_sub.std(axis=1)))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8.5,4), sharex=True, sharey=True)

pt_plot = np.random.binomial(n=1,p=1e-1,size=n_loci*n_loci).astype(bool)
im1 = ax[0].scatter(np.nan_to_num(np.corrcoef(afr_sub).ravel()[pt_plot]), afr_ld.ravel()[pt_plot], s=0.5, c=np.log10(site_distance.ravel()/seq_length)[pt_plot], rasterized=True, vmin=-4.5, vmax=-0.5)
im2 = ax[1].scatter(np.nan_to_num(np.corrcoef(eur_sub).ravel()[pt_plot]), eur_ld.ravel()[pt_plot], s=0.5, c=np.log10(site_distance.ravel()/seq_length)[pt_plot], rasterized=True, vmin=-4.5, vmax=-0.5)

ax[0].plot([-1,1],[-1,1],ls='--',c='black')
ax[1].plot([-1,1],[-1,1],ls='--',c='black')

lim = 0.6 * 1.05
ax[0].set_xlim([-lim,lim])
ax[0].set_ylim([-lim,lim])

fig.text(0.5, -0.02, 'Single-continental LD correlation', fontsize=15, ha='center', va='center')
fig.text(-0.02, 0.5, 'Local ancestry-adjusted LD correlation', fontsize=15, ha='center', va='center', rotation=90)

ax[0].set_title('African', fontsize=15)
ax[1].set_title('European', fontsize=15)

afr_cor = np.corrcoef(np.nan_to_num(np.corrcoef(afr_sub).ravel()), afr_ld.ravel())[0,1]
eur_cor = np.corrcoef(np.nan_to_num(np.corrcoef(eur_sub).ravel()), eur_ld.ravel())[0,1]
ax[0].text(0.1, 0.9, r'Correlation$ = %.2f$' % afr_cor, fontsize=15, transform=ax[0].transAxes)
ax[1].text(0.1, 0.9, r'Correlation$ = %.2f$' % eur_cor, fontsize=15, transform=ax[1].transAxes)

cax = ax[1].inset_axes([1.06, 0, 0.05, 1])
#cax.yaxis.set_label_position("right")
cbar = fig.colorbar(im1, cax=cax)
cbar.ax.set_ylabel('Normalized log-distance (bp) between loci', fontsize=14, rotation=270, labelpad=15)

plt.tight_layout()
plt.savefig('ld_compare.svg', bbox_inches='tight')

In [ ]:
def get_migrating_tracts(ts, anc):
    neanderthal_id = [p.id for p in ts.populations() if p.metadata['name']==anc][0]
    migrating_tracts = []
    # Get all tracts that migrated into the neanderthal population
    for migration in ts.migrations():
        if migration.dest == neanderthal_id:
            migrating_tracts.append((migration.left, migration.right))

    migrating_tracts = np.array(migrating_tracts) 
    return migrating_tracts[:,1] - migrating_tracts[:,0]

In [ ]:
afr_lengths = get_migrating_tracts(mts_common, 'AFR')
eur_lengths = get_migrating_tracts(mts_common, 'EUR')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8,4), sharex=True, sharey=True)

pt_plot = np.random.binomial(n=1,p=1e-1,size=n_loci*n_loci).astype(bool)

ax[0].scatter(np.log10(site_distance.ravel()[pt_plot]/seq_length), np.abs(afr_ld.ravel()[pt_plot]), s=5, label='Local ancestry-adjusted LD correlation', color='red', rasterized=True)
ax[0].hist(np.log10((afr_tracts.right-afr_tracts.left) / seq_length), bins=41, density=True, edgecolor='black',histtype=u'step', linestyle='--', linewidth=1.5, label='Local ancestry length distribution')

ax[1].scatter(np.log10(site_distance.ravel()[pt_plot]/seq_length), np.abs(eur_ld.ravel()[pt_plot]), s=5, color='red', rasterized=True)
ax[1].hist(np.log10((eur_tracts.right-eur_tracts.left) / seq_length), bins=41, density=True, edgecolor='black',histtype=u'step', linestyle='--', linewidth=1.5)

fig.text(0.5, -0.02, 'Normalized log-distance (bp) between loci', fontsize=15, ha='center', va='center')

ax[0].set_title('African', fontsize=15)
ax[1].set_title('European', fontsize=15)


handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, frameon=False, fontsize=15, loc='lower center', bbox_to_anchor=(0.5, 0.97))


plt.tight_layout()
plt.savefig('ld_vs_lanc.svg', bbox_inches='tight')
plt.show()